In [1]:
import os
import sys

In [2]:
sys.path

['C:\\Users\\Linus\\Desktop\\XAI-Vehicle-Classification\\notebooks',
 'C:\\Users\\Linus\\anaconda3\\envs\\mmseg\\python310.zip',
 'C:\\Users\\Linus\\anaconda3\\envs\\mmseg\\DLLs',
 'C:\\Users\\Linus\\anaconda3\\envs\\mmseg\\lib',
 'C:\\Users\\Linus\\anaconda3\\envs\\mmseg',
 '',
 'C:\\Users\\Linus\\anaconda3\\envs\\mmseg\\lib\\site-packages',
 'c:\\users\\linus\\desktop\\mmsegmentation',
 'C:\\Users\\Linus\\anaconda3\\envs\\mmseg\\lib\\site-packages\\win32',
 'C:\\Users\\Linus\\anaconda3\\envs\\mmseg\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\Linus\\anaconda3\\envs\\mmseg\\lib\\site-packages\\Pythonwin']

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
sys.path

['C:\\Users\\Linus\\Desktop\\XAI-Vehicle-Classification\\notebooks',
 'C:\\Users\\Linus\\anaconda3\\envs\\mmseg\\python310.zip',
 'C:\\Users\\Linus\\anaconda3\\envs\\mmseg\\DLLs',
 'C:\\Users\\Linus\\anaconda3\\envs\\mmseg\\lib',
 'C:\\Users\\Linus\\anaconda3\\envs\\mmseg',
 '',
 'C:\\Users\\Linus\\anaconda3\\envs\\mmseg\\lib\\site-packages',
 'c:\\users\\linus\\desktop\\mmsegmentation',
 'C:\\Users\\Linus\\anaconda3\\envs\\mmseg\\lib\\site-packages\\win32',
 'C:\\Users\\Linus\\anaconda3\\envs\\mmseg\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\Linus\\anaconda3\\envs\\mmseg\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\Linus\\Desktop\\XAI-Vehicle-Classification']

In [5]:
from scripts import test

In [8]:
test.references()

From Ref
From Ref2
